## Demo of using PDBrenum to perform mapping of chain IDs in PDB files to UniProt IDs

PDBrenum ([GitHub repo](https://github.com/Faezov/PDBrenum) & associated [2021 publication](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0253411)) is used here as the first step to perform mapping of chain identifiers in PDB files to UniProt IDs. PDBrenum uses information form the [SIFTS project database](https://www.ebi.ac.uk/pdbe/docs/sifts/) to take a PDB structure file renumber chains in PDB files to match the UniProt entries. SIFTS stands for 'Structure integration with function, taxonomy and sequence' and is an up-to-date resource for residue-level mapping between UniProt and PDB entries. PDBrenum handles accessing that information as part of its process and makes several files that summarize the information from there. If you wish, you can look at demonstration of PDBrenum [here](demo.ipynb). That is probably not necessary to understand what is going on here as use of the PDBrenum code is fairly straightforward as it handles all the steps for you, and knowing what it does and that it generates as a side product files summarizing appropriate data you can probably understand why it could be useful as part of an endeavor to mapr chain identifiers to UniProt identifiers.

Needing to map chain identifiers to UniProt identifiers is apparently a common task as it has come up several times, see [here](https://www.biostars.org/p/9540519/#9540519) which contains a reference to other people seeking this ability. Although other modern solutions were offered there and the OP offered code they put together, it's always good to have options.  My suggestion of using PDBrenum to do the first step saves a lot of coding as really the job is already done. You just need to get the data PDBrenum summaizes as part of its effort back into Python where it can be used further. I assume the developers of PDBrenum will have reason to support the code for sometime as they have published about it. 


Compare and contrast with process & result from johnnytam100's [pdb2uniprot](https://github.com/johnnytam100/pdb2uniprot), described [here](https://www.biostars.org/p/9540519/#9540826).


-----

<div class="alert alert-block alert-warning">
<p>If you haven't used one of these notebooks before, they're basically web pages in which you can write, edit, and run live code. They're meant to encourage experimentation, so don't feel nervous. Just try running a few cells and see what happens!.</p>

<p>
    Some tips:
    <ul>
        <li>Code cells have boxes around them.</li>
        <li>To run a code cell, click on the cell and either click the <i class="fa-play fa"></i> button on the toolbar above, or then hit <b>Shift+Enter</b>. The <b>Shift+Enter</b> combo will also move you to the next cell, so it's a quick way to work through the notebook. Selecting from the menu above the toolbar, <b>Cell</b> > <b>Run All</b> is a shortcut to trigger attempting to run all the cells in the notebook.</li>
        <li>While a cell is running a <b>*</b> appears in the square brackets next to the cell. Once the cell has finished running the asterisk will be replaced with a number.</li>
        <li>In most cases you'll want to start from the top of notebook and work your way down running each cell in turn. Later cells might depend on the results of earlier ones.</li>
        <li>To edit a code cell, just click on it and type stuff. Remember to run the cell once you've finished editing.</li>
    </ul>
</p>
</div>

----

Step through running the cells below. Then substitute in your PDB entry identifiers of interest.

------

## Preparation

The packages needed to run PDBrenum are already installed.
Let's also get some input data to use. The source we'll use also provides a script johnnytam100 made to do this task as well, and we'll use it to compare and contrast with that as well.

In [1]:
!git clone https://github.com/johnnytam100/pdb2uniprot.git

Cloning into 'pdb2uniprot'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


Let's move those files, except the README, all to the same place as this notebook. (The README.md will be hidden before the main move commmand is run so it will be left & then it will be unhidden.)

In [2]:
!mv pdb2uniprot/README.md pdb2uniprot/.README.md 
!mv pdb2uniprot/* .
!mv pdb2uniprot/.README.md pdb2uniprot/README.md 
!ls

binder					  pdb2uniprot	       PDBrenum.py
chainID_mapping_to_UniProt_id_demo.ipynb  pdb2uniprot_tam.py   README.md
demo.ipynb				  pdb_chain_table      src
input.txt				  pdb_chain_table.csv
LICENSE					  PDBrenum.ipynb


To see if getting all that worked, let's run the script and see the results. 
For the first input, we'll use `pdb_chain_table.csv` as used in the first option listed at https://github.com/johnnytam100/pdb2uniprot .  
Let's look at the starting point:

In [3]:
cat pdb_chain_table.csv

PDB_ID,CHAIN_ID
7DPA,A
XXXX,X
7DPA,B
7DPA,Z
1BZQ,A
7dpa,z
1bzq,A
1BZQ,a
1bzq,a

Now to actually run the first option listed at https://github.com/johnnytam100/pdb2uniprot :

In [4]:
%run pdb2uniprot_tam.py --input pdb_chain_table.csv --pdb_col PDB_ID --chain_col CHAIN_ID

mapping... 7DPA A
mapping... XXXX X
XXXX X PDB Not Found (HTTP Error 404). Skipped.
mapping... 7DPA B
mapping... 7DPA Z
7DPA Z PDB Found but Chain Not Found. Skipped.
mapping... 1BZQ A
mapping... 7dpa z
7dpa z PDB Found but Chain Not Found. Skipped.
mapping... 1bzq A
mapping... 1BZQ a
mapping... 1bzq a


A new file named `pdb_chain_table_uniprot.csv`, based on the input name, `pdb_chain_table.csv`,  is generated.

In [5]:
ls

binder/                                   pdb_chain_table
chainID_mapping_to_UniProt_id_demo.ipynb  pdb_chain_table.csv
demo.ipynb                                pdb_chain_table_uniprot.csv
input.txt*                                PDBrenum.ipynb*
LICENSE                                   PDBrenum.py*
pdb2uniprot/                              README.md*
pdb2uniprot_tam.py                        src/


In [6]:
cat pdb_chain_table_uniprot.csv

PDB_ID,CHAIN_ID,uniprot
7DPA,A,Q9H7D0
XXXX,X,NaN
7DPA,B,P63000
7DPA,Z,NaN
1BZQ,A,P61823
7dpa,z,NaN
1bzq,A,P61823
1BZQ,a,P61823
1bzq,a,P61823


That's easier to read if we read bring it into Pandas and take advantage of Jupyter's excellent handling of Pandas dataframes to view:

In [7]:
import pandas as pd
jtdf = pd.read_csv("pdb_chain_table_uniprot.csv")
jtdf

,PDB_ID,CHAIN_ID,uniprot
0,7DPA,A,Q9H7D0
1,XXXX,X,NaN
2,7DPA,B,P63000
3,7DPA,Z,NaN
4,1BZQ,A,P61823
5,7dpa,z,NaN
6,1bzq,A,P61823
7,1BZQ,a,P61823
8,1bzq,a,P61823


(Note that chain identifiers are actually case-sensitive, for example [5v2c](https://www.rcsb.org/structure/5V2C) has both chains designated A and a, and so I don't agree with CHAIN_ID of 'A' being same as 'a'.)

`pdb2uniprot_tam.py` will also take tables as tab-separated data:

In [8]:
%run pdb2uniprot_tam.py --input pdb_chain_table

mapping... 7DPA A
mapping... XXXX X
XXXX X PDB Not Found (HTTP Error 404). Skipped.
mapping... 7DPA B
mapping... 7DPA Z
7DPA Z PDB Found but Chain Not Found. Skipped.
mapping... 1BZQ A
mapping... 7dpa z
7dpa z PDB Found but Chain Not Found. Skipped.
mapping... 1bzq A
mapping... 1BZQ a
mapping... 1bzq a


In [9]:
cat pdb_chain_table_uniprot

pdb	chain	uniprot
7DPA	A	Q9H7D0
7DPA	B	P63000
1BZQ	A	P61823
1bzq	A	P61823
1BZQ	a	P61823
1bzq	a	P61823


Before we leave the preparation section, let's create a list of PDB codes for later use. We can use jtdf to do that.

One option is to use:

```python
extracted = jtdf[["PDB_ID"]]
extracted.to_csv("codes.txt", header = False, index = False)
```

But that's going to result in `codes.txt` with several duplicates. Let's make the extraction step more complex to get a simpler list of codes. In the more complex version, we'll make all the codes lowercase and then drop the dupliates

In [10]:
jtdf["PDB_ID"] = jtdf["PDB_ID"].str.lower()
extracted = jtdf[["PDB_ID"]].drop_duplicates()
extracted.to_csv("codes.txt", header = False, index = False)

In [11]:
cat codes.txt

7dpa
xxxx
1bzq


Note that if the file where the PDB codes are stored had just been a single column of PDB ids with each one on a separate line, like we just made with `codes.txt`, then it would have been easy with the following code to read that into Python as a list without invoking Pandas and using only the core Python library:

```python
input_file_name = "codes.txt"
collected_ids = []
with open(input_file_name, 'r') as file_in_stream:
    for line in file_in_stream:
        collected_ids.append(line.strip())
```

I'm providing that here as another option since who knows what form of actual input you made have to deal with and some may find direct Python easier to adapt.  
Because we are dealing with tables to read in the data that's generated as a by-product of running PDBrenum and that data is in text table form, we'll need Pandas installed and so it isn't really feasible to avoid using Pandas in this effort. However, hopefully featuring the option with the core Python library is enough to give you a sense they are options beyond Pandas.

--------

# Quickstart overview: use of PDBrenum to map PDB and chain IDs to UniProt IDs

Now that the preparation steps are complete, this sub-section is meant to quickly show the code and the process to those who are familiar with PDBrenum use/SIFTS data and Python. (For example, you may have come from the Biostars post [Mapping PDB ID + chain ID to UniProt ID](https://www.biostars.org/p/9540519/#9540582) and just want to see the 'typical' route all together.) 

If you know you aren't going to follow it, you may skip running the code in this sub-section and go right onto the next sub-section.

If you want to understand what this code block below is doing or understand the many ways you can adapt it. Or even see varations, read on the further sections below.

This example will process one of the inputs that `pdb2uniprot_tam.py` uses to demonstrate how it works using one of the straighforward processes that are futher explored below in this notebook. Click in the cell and type `shift+enter` to run it:

In [12]:
import pandas as pd
input_df = pd.read_csv("pdb_chain_table.csv")
input_df["PDB_ID"] = input_df["PDB_ID"].str.lower()
codes_list = input_df[["PDB_ID"]].drop_duplicates().PDB_ID.tolist()
codes_list_as_text = " ".join(codes_list)
%run PDBrenum.py -rfla {codes_list_as_text} -PDB
df = pd.read_fwf("log_corrected.txt", )
dfsub = df[['PDB_id', 'chain_PDB','UniProt']]
df_dict = dfsub.groupby('PDB_id').apply(lambda x: dict(zip(x.chain_PDB, x.UniProt))).to_dict()
def lookup_id(items):
    '''
    takes a row with PDB_ID and CHAIN_ID and returns corresponding UniProt ID
    '''
    pdb_id = items[0].lower()
    chain_id = items[1] 
    if pdb_id not in df_dict:
        return 'NA'
    if chain_id not in df_dict[pdb_id]:
        return 'NA'
    uniprot_id = df_dict[pdb_id][chain_id]
    return uniprot_id
result_df = input_df.copy()
result_df['UniProt'] = result_df.apply(lookup_id, axis=1)
result_df.to_csv("result_with_uniprot.tsv",index=False, sep = "\t")

Renumbering PDB files: 100%|██████████| 2/2 [00:07<00:00,  3.97s/it]


To show the results here, run the next cell:

In [13]:
!cat result_with_uniprot.tsv

PDB_ID	CHAIN_ID	UniProt
7dpa	A	Q9H7D0
xxxx	X	NA
7dpa	B	P63000
7dpa	Z	NA
1bzq	A	P61823
7dpa	z	NA
1bzq	A	P61823
1bzq	a	NA
1bzq	a	NA


Read on for more insight or how you can adapt it or code below for your specific use.

-----

Notes on Snakemake and PDBrenum to mapping chain IDs to UniProt IDs
---------------

If you are familiar with any of my other scripts and workflows, you may know I like snakemake for pulling things together into a useable pipeline. So why not here? It certainly could be done. One of the main benefits of snakemake is that upon change in files, say adding input, it will analyze your pipeline for any product it doesn't have have and make just that product. However, to get that benefit here in conjunction with PDBrenum use, it would necessitate handling a lot of steps that aren't already there. Frankly, PDBrenum is set up to be a pipeline in itself. The steps needed to process a lot pod PDB files are straightforward and easy to write/adapt. It seems making it a snakemake wokflow at this time isn't worth the effort.


----

## One way to use PDBrenum to map chain IDs to UniProt IDs: letting PDBrenum handle several PDB codes

Now that the preparation steps are complete, we have input files and any other items needed to work through the rest of this notebook. Let's get started.

How exactly the iterating on different PDB files is handled is one consideration when deciding what way you want to integrate the use of PDBrenum for mapping into your work. And how you handle it might come down to what else you are doing when you need the mappings, and so I'll show first where PDBrenum's handling of multiple PDB identifiers is used and then later we'll iterate on each PDB file, running PDBrenum each time instead of just once.

First we'll need the PDB codes, and so we'll get them from the example input. The example input has two entries on each line: the PDB file identifier and the chain designation. After we use the double columned example input, we'll demonstrate using just the PDB codes.

### A: Using double-column input

We'll use `pdb_chain_table.csv` and `pdb_chain_table` as examples for variations on double-column input. Both input options contain a PDB id and a corresponding chain id on each line, as was shown above. The first is comma delimited and the second is tab delimited.

We'll need the PDB codes that we'll supply to the main PDBrenum command to accomplish the main step. We can use a variation on the first steps we used when we were making `codes.txt` in the preparation section. We'll make a Python list with the PDB id codes for now. The PDB id codes will be in lowercase to standardize them easily so we can remove duplicates or see anythign out of the ordinary more clearly, i.e., less chance of confusing the letter`O` for a zero, etc.. We'll first use the variation on the steps in the preparation section, using a dataframe already define there, to highlight how Pandas makes it easy and to highlight the steps separate from reading in the input data.

In [14]:
import pandas as pd
jtdf["PDB_ID"] = jtdf["PDB_ID"].str.lower()
codes_list = jtdf[["PDB_ID"]].drop_duplicates().PDB_ID.tolist()
codes_list

['7dpa', 'xxxx', '1bzq']

Note that because the dataframe `jtdf` came from a modified version of the text file `pdb_chain_table.csv` and only concerns the column with the PDB id codes, it's equivalent to reading in the PDB id codes from `pdb_chain_table.csv`. Now that I highlighted just those two steps to make it clear, it's best to cover how to start with the input file as that is where you'd be if you were starting with your own PDB id codes and chain identifiers to process



So we'll read in from the actual input file now using Pandas similar to what was done in the preparation and add in the steps just highlighted above. Let's make `codes_list` that way by running the next cell.

In [15]:
import pandas as pd
input_df = pd.read_csv("pdb_chain_table.csv")
input_df["PDB_ID"] = input_df["PDB_ID"].str.lower()
codes_list = input_df[["PDB_ID"]].drop_duplicates().PDB_ID.tolist()
codes_list

['7dpa', 'xxxx', '1bzq']

In [16]:
codes_list_as_text = " ".join(codes_list)
%run PDBrenum.py -rfla {codes_list_as_text} -PDB

Renumbering PDB files: 100%|██████████| 2/2 [00:07<00:00,  3.67s/it]


(The purpose/use of the `-rfla` flag was covered in [Demo of PDBrenum in your broswer via MyBinder.org](demo.ipynb).)

The file created `log_corrected.txt` has the information we need.

In [17]:
cat log_corrected.txt

SP PDB_id chain_PDB   chain_auth  UniProt             SwissProt              uni_len chain_len     renum 5k_or_50k
+  7dpa   A           A           Q9H7D0              DOCK5_HUMAN               1642      1642         0         0
+  7dpa   B           B           P63000              RAC1_HUMAN                 177       177         0         0
+  7dpa   C           C           Q92556              ELMO1_HUMAN                198       198         0         0
+  7dpa   D           D           Q9H7D0              DOCK5_HUMAN               1642      1642         0         0
+  7dpa   E           E           P63000              RAC1_HUMAN                 177       177         0         0
+  7dpa   F           F           Q92556              ELMO1_HUMAN                198       198         0         0
+  1bzq   A           A           P61823              RNAS1_BOVIN                124       124       124         0
+  1bzq   B           B           P61823              RNAS1_BOVIN               

The table in `cat log_corrected.txt` shown above is a fixed-width format text table that Pandas can read in.  
We can then use that Pandas dataframe to organize the appropriate data for ease of access as we try to relate to the input lines. The organizaing consists of making a dictionary of dictionaries with the assgnments per PDB file. The PDB codes will be the keys of the main overarching dictionary. Then in that overarching dictionary for each PDB code the values contained will be dictionary, with the chain designations as keys and the corresponding UniProt id as the value.  

In [18]:
import pandas as pd
df = pd.read_fwf("log_corrected.txt", ) # based on https://stackoverflow.com/a/41509522/8508004
# We only need the three columns that have 'PDB_id', 'chain_PDB', and 'UniProt'
dfsub = df[['PDB_id', 'chain_PDB','UniProt']]
#df_dict = dfsub.to_dict(orient='records') # If you prefer each row as a dictionary
#df_dict = dfsub.groupby('PDB_id').apply(lambda x: [dict(zip(x.chain_PDB, x.UniProt))]).to_dict() # based on https://stackoverflow.com/a/41064974/8508004; 
# it makes a dictionary of a list of dictionaries
df_dict = dfsub.groupby('PDB_id').apply(lambda x: dict(zip(x.chain_PDB, x.UniProt))).to_dict() # based on https://stackoverflow.com/a/41064974/8508004
# {k: [v.to_dict()] for k, v in dfsub.set_index(['PDB_id', 'chain_PDB']).UniProt.unstack(0).iteritems()}  # based on https://stackoverflow.com/a/41065429/8508004;
# it makes a dictionary of a list of dictionaries but note that it tries to make all sub dictionaries have same chain elementes it seems and so puts `nan` for chains that don't have UniProt id values
#{k: v.to_dict() for k, v in dfsub.set_index(['PDB_id', 'chain_PDB']).UniProt.unstack(0).iteritems()}}  # based on https://stackoverflow.com/a/41065429/8508004;
# but see caveat about chain elements above the dictionary comprehenseion
df_dict

{'1bzq': {'A': 'P61823', 'B': 'P61823', 'C': 'P61823', 'D': 'P61823'},
 '7dpa': {'A': 'Q9H7D0',
  'B': 'P63000',
  'C': 'Q92556',
  'D': 'Q9H7D0',
  'E': 'P63000',
  'F': 'Q92556'}}

With the assignments per PDB file, we can read in the input file and then use that to make a table with the UniProt id added. (Because I gave options for getting the PDB codes earlier for the above steps, I'm not outright assuming `input_df` was already defined.) First, we'll do that with the comma delimited version:

In [19]:
import pandas as pd
input_df = pd.read_csv("pdb_chain_table.csv")
def lookup_id(items):
    '''
    takes a row with PDB_ID and CHAIN_ID and returns corresponding UniProt ID
    '''
    pdb_id = items[0].lower()
    chain_id = items[1] 
    if pdb_id not in df_dict:
        return 'NA'
    if chain_id not in df_dict[pdb_id]:
        return 'NA'
    uniprot_id = df_dict[pdb_id][chain_id]
    return uniprot_id
result_df = input_df.copy()
result_df['UniProt'] = result_df.copy().apply(lookup_id, axis=1)

In [20]:
result_df

,PDB_ID,CHAIN_ID,UniProt
0,7DPA,A,Q9H7D0
1,XXXX,X,NA
2,7DPA,B,P63000
3,7DPA,Z,NA
4,1BZQ,A,P61823
5,7dpa,z,NA
6,1bzq,A,P61823
7,1BZQ,a,NA
8,1bzq,a,NA


The same main code will work for the tab-delimited data. The only change needed is specifying the delimiter for reading in to Python (Pandas type data, specifically). We'll tell Pandas the delimiter is a tab, which is signaled with `/t`. The tab-delimited input file provided by  [Johnny Tam's pdb2uniprot repository](https://github.com/johnnytam100/pdb2uniprot), that we are using as a source of example data, also lacks the column headings and so we need to add those when reading the tab-delimited file. (Note that is the file `pdb_chain_table` had an extension of `.tsv` it would be more obvious and indeed Jupyter would provide conevenience viewers that work with it, but it's not necessary since we know from [Johnny Tam's pdb2uniprot repository](https://github.com/johnnytam100/pdb2uniprot) that it is the 'tab-delimited table').

In [21]:
## Using tab delimited input
import pandas as pd
input_df = pd.read_csv("pdb_chain_table", sep="\t", names = ["PDB_ID","CHAIN_ID"])
def lookup_id(items):
    '''
    takes a row with PDB_ID and CHAIN_ID and returns corresponding UniProt ID
    '''
    pdb_id = items[0].lower()
    chain_id = items[1] 
    if pdb_id not in df_dict:
        return 'NA'
    if chain_id not in df_dict[pdb_id]:
        return 'NA'
    uniprot_id = df_dict[pdb_id][chain_id]
    return uniprot_id
result_df = input_df.copy()
result_df['UniProt'] = result_df.apply(lookup_id, axis=1)
result_df

,PDB_ID,CHAIN_ID,UniProt
0,7DPA,A,Q9H7D0
1,XXXX,X,NA
2,7DPA,B,P63000
3,7DPA,Z,NA
4,1BZQ,A,P61823
5,7dpa,z,NA
6,1bzq,A,P61823
7,1BZQ,a,NA
8,1bzq,a,NA


So the only real change relative the code showed earlier is this line:

```python
input_df = pd.read_csv("pdb_chain_table", sep="\t", names = ["PDB_ID","CHAIN_ID"])
```

Note that pdb2uniprot makes the output as a data text table file with the choice of having the header or not.  
That can easily be done with the Pandas dataframe we've made with the two types of input. Demonstrating that with the current dataframe in memory, first we'll save the output file with the header (`index=False` means we don't want the row numbers that the dataframe has, which are a Pandas `index` object):

In [22]:
result_df.to_csv("result_with_uniprot.csv",index=False)

We can show that worked by looking at the contents here by running the following code:

In [23]:
!cat result_with_uniprot.csv

PDB_ID,CHAIN_ID,UniProt
7DPA,A,Q9H7D0
XXXX,X,NA
7DPA,B,P63000
7DPA,Z,NA
1BZQ,A,P61823
7dpa,z,NA
1bzq,A,P61823
1BZQ,a,NA
1bzq,a,NA


We can make that tab-delimited if we prefer by specifying the separator as we did when reading in the tab-separated input:

In [24]:
result_df.to_csv("result_with_uniprot.tsv",index=False, sep = "\t")

Verifying that here:

In [25]:
!cat result_with_uniprot.tsv

PDB_ID	CHAIN_ID	UniProt
7DPA	A	Q9H7D0
XXXX	X	NA
7DPA	B	P63000
7DPA	Z	NA
1BZQ	A	P61823
7dpa	z	NA
1bzq	A	P61823
1BZQ	a	NA
1bzq	a	NA


As I mentioned above, Johnny Tam's pdb2uniprot makes the output as a data text table file with the choice of not having the header. That can easily be done with the Pandas dataframe by including `header = False` in the call to the method:

In [26]:
result_df.to_csv("result_with_uniprot_no_header.tsv",index=False, sep = "\t", header = False)

In [27]:
!cat result_with_uniprot_no_header.tsv

7DPA	A	Q9H7D0
XXXX	X	NA
7DPA	B	P63000
7DPA	Z	NA
1BZQ	A	P61823
7dpa	z	NA
1bzq	A	P61823
1BZQ	a	NA
1bzq	a	NA


### B: Using a list of PDB codes

Imagine you just had a list of PDB codes and you wanted the information for all chains present. We'll use the list of codes we made in the the last part of the preparation section to demonstrate this situation.

In [28]:
# `codes.txt` was made in the preparation section; as a reminder. Here are the contents:
!cat codes.txt

7dpa
xxxx
1bzq


This approach really follows from the beginning of the section 'A' just covered. Because we want all the results, we don't need to look up anything and we can just use the reduced version of all the data in `log_corrected.txt`.

In [29]:
import pandas as pd
df_simple = pd.read_csv('codes.txt', header=None, names = ['PDB_ids'])
codes_list = df_simple['PDB_ids'].to_list()
codes_list_as_text = " ".join(codes_list)
%run PDBrenum.py -rfla {codes_list_as_text} -PDB
df = pd.read_fwf("log_corrected.txt", )
df_results = df[['PDB_id', 'chain_PDB','UniProt']]
df_results.to_csv("all_result_with_uniprot.tsv",index=False, sep = "\t")
df_results

Renumbering PDB files: 100%|██████████| 2/2 [00:07<00:00,  3.79s/it]


,PDB_id,chain_PDB,UniProt
0,7dpa,A,Q9H7D0
1,7dpa,B,P63000
2,7dpa,C,Q92556
3,7dpa,D,Q9H7D0
4,7dpa,E,P63000
5,7dpa,F,Q92556
6,1bzq,A,P61823
7,1bzq,B,P61823
8,1bzq,C,P61823
9,1bzq,D,P61823


We included making a text file above for options for downstream use. Running the next cell will show the contents here:

In [30]:
!cat all_result_with_uniprot.tsv

PDB_id	chain_PDB	UniProt
7dpa	A	Q9H7D0
7dpa	B	P63000
7dpa	C	Q92556
7dpa	D	Q9H7D0
7dpa	E	P63000
7dpa	F	Q92556
1bzq	A	P61823
1bzq	B	P61823
1bzq	C	P61823
1bzq	D	P61823


It will look better if you double-click on `all_result_with_uniprot.tsv` listed in the file browser on the left and open it with JupyterLab's TSV viewer, and then return here.

**Read in the input file line per line without Pandas.**  
Since we were already usng Pandas, I simply used Pandas to read in the input data, too. However, as I pointed out earlier in this notebook theres other routes to read such a single-column, like-like text file with an item on each line. For example, it would have been easy with the following code to read `codes.txt` into Python as a list without invoking Pandas and using only the core Python library:

```python
input_file_name = "codes.txt"
collected_ids = []
with open(input_file_name, 'r') as file_in_stream:
    for line in file_in_stream:
        collected_ids.append(line.strip())
```

Plugging that in instead of using Pandas, the code just demonstrate above to get all the chains and ID mapping for each PDB would be:

In [31]:
import pandas as pd
input_file_name = "codes.txt"
collected_ids = []
with open(input_file_name, 'r') as file_in_stream:
    for line in file_in_stream:
        collected_ids.append(line.strip())
codes_list_as_text = " ".join(collected_ids)
%run PDBrenum.py -rfla {codes_list_as_text} -PDB
dfB = pd.read_fwf("log_corrected.txt", )
df_resultsB = df[['PDB_id', 'chain_PDB','UniProt']]
df_resultsB.to_csv("alt_all_result_with_uniprot.tsv",index=False, sep = "\t")
df_resultsB

Renumbering PDB files: 100%|██████████| 2/2 [00:08<00:00,  4.13s/it]


,PDB_id,chain_PDB,UniProt
0,7dpa,A,Q9H7D0
1,7dpa,B,P63000
2,7dpa,C,Q92556
3,7dpa,D,Q9H7D0
4,7dpa,E,P63000
5,7dpa,F,Q92556
6,1bzq,A,P61823
7,1bzq,B,P61823
8,1bzq,C,P61823
9,1bzq,D,P61823


In [32]:
!cat alt_all_result_with_uniprot.tsv

PDB_id	chain_PDB	UniProt
7dpa	A	Q9H7D0
7dpa	B	P63000
7dpa	C	Q92556
7dpa	D	Q9H7D0
7dpa	E	P63000
7dpa	F	Q92556
1bzq	A	P61823
1bzq	B	P61823
1bzq	C	P61823
1bzq	D	P61823


## Another way to use PDBrenum to map chain IDs to UniProt IDs: iterating through running PDBrenum with one PDB code at a time and further mining the generated by by-product information in turn

Using PDBrenum as it was above means it handles multiple PDB ids by default and handles issues with codes that aren't true PBB codes without you needing to handle that aspect. Those are niceties that make the script user-friendly that you don't need to worry about. **Plus, when you can it makes more sense to rely on code someone has written and released rather then writing everything yourself.** Code that you don't need to write or maintain that does what you need is a gift.

However, in the itnerest of options, you may wish to only run PDBrnum on individual PDB files each time and process the data in `log_corrected.txt` for each individual PDB id and then combine the details subsequently. For example, you may be doing something else with the PDB file or PDB id in parallel that you want to combine the mappng of the chain ID to UniProt as well. Or the data you want to mine from PDBrenum results or `log_corrected.txt` turns out to be easier when there's only one PDB id being processed at a time. This section will show how to run the PDBrenum script with each PDB id and get the mapping of the chain IDs to UniProt IDs out.

We'll do that with both the double-column input type and the single column to match with sections **A** and **B** above.

### A: Using double-column input and iterating on each PDB id code running PDBrenum one at time

First we'll adapt the typical route listed in the Overivew and actually encountered first in this notbeook. So essentially instead of `%run PDBrenum.py -rfla {codes_list_as_text} -PDB` to supply all the PDB ids at once and then using that as a block, we'll loop on each like this:

```python
for pdb_id in codes_list:
    %run PDBrenum.py {pdb_id}
    #...process by-products of pdbrenum with out goal in mind...#
```

Run the next cell to see this in action:

In [33]:
import pandas as pd
input_df = pd.read_csv("pdb_chain_table.csv")
input_df["PDB_ID"] = input_df["PDB_ID"].str.lower()
codes_list = input_df[["PDB_ID"]].drop_duplicates().PDB_ID.tolist()
df_dict = {}
for pdb_id in codes_list:
    %run PDBrenum.py -rfla {pdb_id}
    df = pd.read_fwf("log_corrected.txt", )
    print(df)
    dfsub = df[['PDB_id', 'chain_PDB','UniProt']]
    df_dict_single = dfsub.groupby('PDB_id').apply(lambda x: dict(zip(x.chain_PDB, x.UniProt))).to_dict()
    print(df_dict_single)
    try: # this `try/except` clause will allow for bad PDB id codes by skipping trying to add them since fails when empty
        df_dict[pdb_id] = df_dict_single[pdb_id]
    except KeyError:
        pass
def lookup_id(items):
    '''
    takes a row with PDB_ID and CHAIN_ID and returns corresponding UniProt ID
    '''
    pdb_id = items[0].lower()
    chain_id = items[1] 
    if pdb_id not in df_dict:
        return 'NA'
    if chain_id not in df_dict[pdb_id]:
        return 'NA'
    uniprot_id = df_dict[pdb_id][chain_id]
    return uniprot_id
result_each_code_df = input_df.copy()
result_each_code_df['UniProt'] = result_each_code_df.apply(lookup_id, axis=1)
result_each_code_df.to_csv("one_at_a_time_result_with_uniprot.tsv",index=False, sep = "\t")
result_each_code_df

Checking mmCIF files: 100%|██████████| 1/1 [00:00<00:00,  1.36it/s]

  SP PDB_id chain_PDB chain_auth UniProt    SwissProt  uni_len  chain_len  \
0  +   7dpa         A          A  Q9H7D0  DOCK5_HUMAN     1642       1642   
1  +   7dpa         B          B  P63000   RAC1_HUMAN      177        177   
2  +   7dpa         C          C  Q92556  ELMO1_HUMAN      198        198   
3  +   7dpa         D          D  Q9H7D0  DOCK5_HUMAN     1642       1642   
4  +   7dpa         E          E  P63000   RAC1_HUMAN      177        177   
5  +   7dpa         F          F  Q92556  ELMO1_HUMAN      198        198   

   renum  5k_or_50k  
0      0          0  
1      0          0  
2      0          0  
3      0          0  
4      0          0  
5      0          0  
{'7dpa': {'A': 'Q9H7D0', 'B': 'P63000', 'C': 'Q92556', 'D': 'Q9H7D0', 'E': 'P63000', 'F': 'Q92556'}}



Renumbering mmCIF files: 0it [00:00, ?it/s]
Checking mmCIF files: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]

Empty DataFrame
Columns: [SP, PDB_id, chain_PDB, chain_auth, UniProt, SwissProt, uni_len, chain_len, renum, 5k_or_50k]
Index: []
{}


Checking mmCIF files: 100%|██████████| 2/2 [00:00<00:00,  2.03it/s]

  SP PDB_id chain_PDB chain_auth UniProt    SwissProt  uni_len  chain_len  \
0  +   1bzq         A          A  P61823  RNAS1_BOVIN      124        124   
1  +   1bzq         B          B  P61823  RNAS1_BOVIN      124        124   
2  +   1bzq         C          C  P61823  RNAS1_BOVIN      124        124   
3  +   1bzq         D          D  P61823  RNAS1_BOVIN      124        124   

   renum  5k_or_50k  
0    124          0  
1    124          0  
2    124          0  
3    124          0  
{'1bzq': {'A': 'P61823', 'B': 'P61823', 'C': 'P61823', 'D': 'P61823'}}


,PDB_ID,CHAIN_ID,UniProt
0,7dpa,A,Q9H7D0
1,xxxx,X,NA
2,7dpa,B,P63000
3,7dpa,Z,NA
4,1bzq,A,P61823
5,7dpa,z,NA
6,1bzq,A,P61823
7,1bzq,a,NA
8,1bzq,a,NA


In [34]:
df_dict

{'7dpa': {'A': 'Q9H7D0',
  'B': 'P63000',
  'C': 'Q92556',
  'D': 'Q9H7D0',
  'E': 'P63000',
  'F': 'Q92556'},
 '1bzq': {'A': 'P61823', 'B': 'P61823', 'C': 'P61823', 'D': 'P61823'}}

In [35]:
!cat one_at_a_time_result_with_uniprot.tsv

PDB_ID	CHAIN_ID	UniProt
7dpa	A	Q9H7D0
xxxx	X	NA
7dpa	B	P63000
7dpa	Z	NA
1bzq	A	P61823
7dpa	z	NA
1bzq	A	P61823
1bzq	a	NA
1bzq	a	NA


Same results; however, the larger area highligthed in pink which is coming from the numerous uses of PDBrenum starts to give a sense already this is a less effficient route. However, it's only slightly less and accepting some inefficiency of this step alone may not add much time at all and allow more easily adapting the code to the workflow you need.

### B: Using a list of PDB codes and iterating on each PDB id code running PDBrenum one at time

Section 'B: Using a list of PDB codes' above covered two ways to read in the list of PDB id codes. We'll just use Pandas to read in here. If you want to use basic Python, you can see how the code in that section was adapted and adapt the version here.

Using the first example in Section 'B: Using a list of PDB codes', above, we'll adapt in iterating on the individual PDB code ids and running PDBrenum for each. That produces data for each that we'll read into a dataframe for each as we iterate and them combine all those to produce one for all the input PDBs.

In [36]:
import pandas as pd
df_simple = pd.read_csv('codes.txt', header=None, names = ['PDB_ids'])
codes_list = df_simple['PDB_ids'].to_list()
dfs_collected = []
df_dict = {} # this will only be used here for screening bad PDB id codes that make empty dataframes
for pdb_id in codes_list:
    %run PDBrenum.py -rfla {pdb_id}
    df = pd.read_fwf("log_corrected.txt", )
    dfsub = df[['PDB_id', 'chain_PDB','UniProt']]
    df_dict_single = dfsub.groupby('PDB_id').apply(lambda x: dict(zip(x.chain_PDB, x.UniProt))).to_dict()
    try: # this `try/except` will allow for bad PDB id codes by skipping trying to add anything concerning them since dictionary lookup fails when empty
        df_dict[pdb_id] = df_dict_single[pdb_id]
        dfs_collected.append(dfsub)
    except KeyError:
        pass
df_results = pd.concat(dfs_collected, ignore_index= True)
df_results.to_csv("all_result_with_uniprot.tsv",index=False, sep = "\t")
df_results

Checking mmCIF files: 100%|██████████| 2/2 [00:01<00:00,  1.99it/s]
Renumbering mmCIF files: 0it [00:00, ?it/s]
Checking mmCIF files: 100%|██████████| 2/2 [00:00<00:00,  2.09it/s]


,PDB_id,chain_PDB,UniProt
0,7dpa,A,Q9H7D0
1,7dpa,B,P63000
2,7dpa,C,Q92556
3,7dpa,D,Q9H7D0
4,7dpa,E,P63000
5,7dpa,F,Q92556
6,1bzq,A,P61823
7,1bzq,B,P61823
8,1bzq,C,P61823
9,1bzq,D,P61823


Notes on Snakemake and PDBrenum to mapping chain IDs to UniProt IDs
---------------

If you are familiar with any of my other scripts and workflows, you may know I like snakemake for pulling things together into a useable pipeline. So why not here? It certainly could be done. One of the main benefits of snakemake is that upon change in files, say adding input, it will analyze your pipeline for any product it doesn't have have and make just that product. However, to get that benefit here in conjunction with PDBrenum use, it would necessitate handling a lot of steps that aren't already there. Frankly, PDBrenum is set up to be a pipeline in itself. The steps needed to process a lot pod PDB files are straightforward and easy to write/adapt. It seems making it a snakemake wokflow at this time isn't worth the effort.

--------

Enjoy!



----

----